In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report

In [12]:
df = pd.read_csv('../datasets/Cirrhosis.csv')
df.dropna(inplace=True)
df = df.drop(columns=['ID', 'N_Days', 'Status', 'Drug'])

In [13]:
X = df.drop(columns=['Stage'])
y = df['Stage']

categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = X.select_dtypes(include=['number']).columns.tolist()


categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

X_processed = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Define models
logistic_model = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(multi_class='ovr'))])

neural_network_model = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('classifier', MLPClassifier(max_iter=1000))])

random_forest_model = Pipeline(steps=[('preprocessor', preprocessor),
                                       ('classifier', RandomForestClassifier())])

# Define parameter grids for hyperparameter tuning
param_grid_logistic = {
    'classifier__C': [0.1, 1, 10],
    'classifier__solver': ['liblinear', 'saga']
}

param_grid_nn = {
    'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'classifier__alpha': [0.0001, 0.001]
}

param_grid_rf = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_features': ['auto', 'sqrt'],
    'classifier__min_samples_split': [2, 5, 10]
}

In [15]:
grid_search_logistic = GridSearchCV(logistic_model, param_grid_logistic, cv=5)
grid_search_nn = GridSearchCV(neural_network_model, param_grid_nn, cv=5)
grid_search_rf = GridSearchCV(random_forest_model, param_grid_rf, cv=5)

grid_search_logistic.fit(X_train, y_train)
grid_search_nn.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)

y_pred_logistic = grid_search_logistic.predict(X_test)
y_pred_nn = grid_search_nn.predict(X_test)
y_pred_rf = grid_search_rf.predict(X_test)

print("Logistic Regression:")
print(classification_report(y_test, y_pred_logistic))

print("Neural Network:")
print(classification_report(y_test, y_pred_nn))

print("Random Forest:")
print(classification_report(y_test, y_pred_rf))

c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\moeed\AppD

Logistic Regression:
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         1
         2.0       0.00      0.00      0.00        11
         3.0       0.45      0.54      0.49        24
         4.0       0.61      0.70      0.65        20

    accuracy                           0.48        56
   macro avg       0.26      0.31      0.29        56
weighted avg       0.41      0.48      0.44        56

Neural Network:
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         1
         2.0       0.17      0.18      0.17        11
         3.0       0.50      0.42      0.45        24
         4.0       0.64      0.70      0.67        20

    accuracy                           0.46        56
   macro avg       0.33      0.32      0.32        56
weighted avg       0.47      0.46      0.47        56

Random Forest:
              precision    recall  f1-score   support

         1.0       0.00

c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [16]:
# Ensemble methods
voting_clf = VotingClassifier(estimators=[
    ('logistic', grid_search_logistic.best_estimator_),
    ('nn', grid_search_nn.best_estimator_),
    ('rf', grid_search_rf.best_estimator_)
], voting='soft')

voting_clf.fit(X_train, y_train)

# Evaluate ensemble model
y_pred_voting = voting_clf.predict(X_test)
print("Ensemble Voting Classifier:")
print(classification_report(y_test, y_pred_voting))

c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Ensemble Voting Classifier:
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         1
         2.0       0.19      0.27      0.22        11
         3.0       0.41      0.29      0.34        24
         4.0       0.65      0.75      0.70        20

    accuracy                           0.45        56
   macro avg       0.31      0.33      0.32        56
weighted avg       0.45      0.45      0.44        56



c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\moeed\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo